<a href="https://colab.research.google.com/github/uzaramen108/fenicsx-colab/blob/main/workshop/fenics_workshop_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# setup

In [30]:
# --------------------------------------------------
# 1️⃣ Mount Google Drive (optional, for cache)
# --------------------------------------------------
from google.colab import drive
import os

if not os.path.ismount("/content/drive"):
    drive.mount("/content/drive")
else:
    print("📦 Google Drive already mounted")

# --------------------------------------------------
# 2️⃣ Clone fenicsx-colab repository (idempotent)
# --------------------------------------------------
from pathlib import Path
import subprocess

REPO_URL = "https://github.com/seoultechpse/fenicsx-colab.git"
ROOT = Path("/content")
REPO_DIR = ROOT / "fenicsx-colab"

def run(cmd):
    #print("$", " ".join(map(str, cmd)))
    subprocess.run(cmd, check=True)

if not REPO_DIR.exists():
    print("📥 Cloning fenicsx-colab...")
    run(["git", "clone", REPO_URL, str(REPO_DIR)])
elif not (REPO_DIR / ".git").exists():
    raise RuntimeError("Directory exists but is not a git repository")
else:
    print("📦 Repository already exists — skipping clone")

# --------------------------------------------------
# 3️⃣ Run setup_fenicsx.py IN THIS KERNEL (CRITICAL)
# --------------------------------------------------
print("🚀 Running setup_fenicsx.py in current kernel")

USE_CLEAN = False  # <--- Set True to remove existing environment
opts = "--clean" if USE_CLEAN else ""

get_ipython().run_line_magic(
    "run", f"{REPO_DIR / 'setup_fenicsx.py'} {opts}"
)

# ==================================================
# 4️⃣ Sanity check
# ==================================================
try:
    get_ipython().run_cell_magic('fenicsx', '--info -np 4', '')
except Exception as e:
    print("⚠️ %%fenicsx magic not found:", e)

📦 Google Drive already mounted
📦 Repository already exists — skipping clone
🚀 Running setup_fenicsx.py in current kernel
📦 Google Drive detected — using persistent cache
🔧 Installing FEniCSx environment...
✨ Loading FEniCSx Jupyter magic... %%fenicsx registered

🐍 Python          : 3.11.14
📦 dolfinx         : 0.10.0
💻 Platform        : Linux-6.6.105+-x86_64-with-glibc2.35
🧵 Running as root : True

🔎 fenicsx runtime info
-----------------------
Environment        : fenicsx
micromamba         : /content/micromamba/bin/micromamba
MPI implementation : OPENMPI
MPI version        : mpiexec (OpenRTE) 4.1.2
MPI ranks (-np)    : 4


## colab 내 파일 정리

In [35]:
import os
import glob
import shutil

def clean_workspace():
    print("🧹 작업 공간 청소를 시작합니다...")

    extensions_to_delete = [ # 지울 데이터 리스트
        "*.xdmf", "*.h5",
        "*.vtu", "*.pvd",
        "*.zip",
        "*.png", "*.jpg",
        "*.json"
    ]

    deleted_files_count = 0
    for pattern in extensions_to_delete:
        files = glob.glob(pattern)
        for file in files:
            try:
                os.remove(file)
                deleted_files_count += 1
            except OSError as e:
                print(f"❌ 삭제 실패: {file} - {e}")

    deleted_folders_count = 0
    for folder in folders_to_delete:
        if os.path.exists(folder):
            try:
                shutil.rmtree(folder) # 폴더 내부까지 싹 지움
                deleted_folders_count += 1
                print(f"🗑️ 폴더 삭제됨: {folder}/")
            except OSError as e:
                print(f"❌ 폴더 삭제 실패: {folder} - {e}")

    print("-" * 30)
    print(f"✨ 청소 완료! 총 {deleted_files_count}개의 파일과 {deleted_folders_count}개의 폴더를 삭제했습니다.")

clean_workspace() # 함수 실행

🧹 작업 공간 청소를 시작합니다...
------------------------------
✨ 청소 완료! 총 4개의 파일과 0개의 폴더를 삭제했습니다.
   이제 작업 공간이 깨끗합니다.


# 실행 파일

In [ ]:
%%fenicsx -np 4 --time
from mpi4py import MPI
import dolfinx
from dolfinx import io
import numpy as np

def save_separate_meshes(N_coarse: int, N_fine: int, degree: int):
    comm = MPI.COMM_WORLD

    def g(x):
        return x[0] + np.sin(np.pi * x[0]) * np.cos(3 * np.pi * x[0])

    # -------------------------------------------------------
    # 1. 듬성듬성한 메쉬 (Coarse) -> 파란색 꺾은선 그래프용
    # -------------------------------------------------------
    mesh_c = dolfinx.mesh.create_unit_interval(comm, N_coarse)
    V_c = dolfinx.fem.functionspace(mesh_c, ("Lagrange", degree))

    u_approx = dolfinx.fem.Function(V_c)
    u_approx.interpolate(g)
    u_approx.name = "Approximation_Coarse"

    with io.XDMFFile(comm, "result_coarse.xdmf", "w", encoding=io.XDMFFile.Encoding.HDF5) as xdmf:
        xdmf.write_mesh(mesh_c)
        xdmf.write_function(u_approx)

    # -------------------------------------------------------
    # 2. 아주 촘촘한 메쉬 (Fine) -> 빨간색 부드러운 곡선용
    # -------------------------------------------------------
    mesh_f = dolfinx.mesh.create_unit_interval(comm, N_fine)
    V_f = dolfinx.fem.functionspace(mesh_f, ("Lagrange", degree))

    u_exact = dolfinx.fem.Function(V_f)
    u_exact.interpolate(g)
    u_exact.name = "Exact_Fine"

    with io.XDMFFile(comm, "result_fine.xdmf", "w", encoding=io.XDMFFile.Encoding.HDF5) as xdmf:
        xdmf.write_mesh(mesh_f)
        xdmf.write_function(u_exact)

save_separate_meshes(N_coarse=10, N_fine=200, degree=1) # 실행

if MPI.COMM_WORLD.rank == 0:
    print("\n✅ 두 개의 파일이 생성되었습니다.")
    print("   1. result_coarse.xdmf (파란색용)")
    print("   2. result_fine.xdmf (빨간색용)")